## __文章要約__

### __準備__

In [1]:
!nvidia-smi

Mon Oct 23 03:40:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install datasets transformers[ja,torch] sentencepiece | tail -n1

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq

### __前処理__

In [4]:
# T5
model_name = "retrieva-jp/t5-base-long"

In [5]:
# Livedoorデータ
dataset = load_dataset("llm-book/livedoor-news-corpus")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'date', 'title', 'content', 'category'],
        num_rows: 5893
    })
    validation: Dataset({
        features: ['url', 'date', 'title', 'content', 'category'],
        num_rows: 736
    })
    test: Dataset({
        features: ['url', 'date', 'title', 'content', 'category'],
        num_rows: 738
    })
})

In [19]:
# トークナイザ
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 前処理
def preprocess_data(data: dict):
    # 入力データ
    inputs = tokenizer(
        data["content"],
        max_length=512,
        truncation=True,
        padding='max_length',
    )

    # 正解データ
    inputs["labels"] = tokenizer(
        data["title"],
        max_length=128,
        truncation=True,
        padding='max_length',
    )["input_ids"]

    return inputs

train_dataset = dataset["train"].map(
    preprocess_data,
    remove_columns=dataset["train"].column_names,
)

validation_dataset = dataset["validation"].map(
    preprocess_data,
    remove_columns=dataset["validation"].column_names,
)

test_dataset = dataset["test"].map(
    preprocess_data,
    remove_columns=dataset["test"].column_names,
)

## __学習__

In [8]:
# モデル
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# data_collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)

In [10]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,17.856000,1.290383
2,1.424200,0.453332
3,0.578200,0.379450
4,0.505100,0.364156
5,0.459800,0.360527


TrainOutput(global_step=3685, training_loss=3.0523025388471794, metrics={'train_runtime': 5552.1706, 'train_samples_per_second': 5.307, 'train_steps_per_second': 0.664, 'total_flos': 2.017637538988032e+16, 'train_loss': 3.0523025388471794, 'epoch': 5.0})